## Transformations used to convert original dataset to match SOPHY

In [5]:
import pandas as pd
import numpy as np
import sqlite3
import geolabel
import sophysql
import sophytaxa
import cartopy.crs as ccrs
import geopandas as gpd
from geopandas import GeoDataFrame
from pandas import DataFrame

In [6]:
con = sqlite3.connect("test.db")
with open('schema.sql', 'r') as sql_file:
    con.executescript(sql_file.read())
con.commit()

1) Palmer LTER dataset
- Link to source and info

In [ ]:
lter1 = pd.read_csv("../data/in/datasets/unmodified/AntarcticaLTERcompiledData_Cruise_forEDI.csv")
lter2 = pd.read_csv("../data/in/datasets/unmodified/AntarcticaLTERcompiledData_Station_forEDI.csv")

lter_sql: dict = {"DatetimeGMT": "timestamp", "Latitude": "latitude", "Longitude": "longitude",
                  "Depth": "depth", "Temperature": "temperature", "Salinity": "salinity", "Density": "density",
                  "Chlorophyll": "chl_a", "Fluorescence": "fluorescence", "Phaeopigment": "phaeopigments",
                  "PrimaryProduction": "primary_prod", "studyName": "cruise", "PAR": "par",
                  "Prasinophytes": "prasinophytes", "Cryptophytes": "cryptophytes",
                  "MixedFlagellates": "mixed_flagellates", "Diatoms": "diatoms", "Haptophytes": "haptophytes",
                  "NO3": "nitrate", "NO2": "nitrite", "DIC1": "diss_inorg_carbon", "DOC": "diss_org_carbon",
                  "POC": "part_org_carbon", "SiO4": "silicate", "N": "tot_nitrogen",
                  "PO4": "phosphate", "Notes1": "notes"}

lter1 = lter1[lter_sql.keys()].rename(columns=lter_sql)
lter2 = lter2[lter_sql.keys()].rename(columns=lter_sql)
lter = pd.concat([lter1, lter2])
lter = lter.dropna(subset=['timestamp', 'longitude', 'latitude'])
# TODO: warning if any values were dropped
lter = lter[lter['longitude'].between(-180, 180)]
lter = lter[lter['latitude'] <= -30]
# Group chemtax into three main categories
lter['percent_phaeo'] = lter['haptophytes']
lter['percent_diatom'] = lter['diatoms']
lter['percent_prasinophytes'] = lter['prasinophytes']
lter['percent_other'] = lter['mixed_flagellates'] + lter['cryptophytes']

data_gdf = GeoDataFrame(lter, geometry=gpd.points_from_xy(lter['longitude'], lter['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(geolabel.zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labelled data
lter = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right']))
sectors_series: pd.Series = pd.cut(lter['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
lter = lter.assign(sector=sectors_series)

extra = lter.columns.difference(sophysql.get_table_cols("sample"))
lter["extra_json"] = lter[extra].agg(lambda r: r[r.notna()].to_json(), axis=1)
lter = lter.drop(extra, axis=1)
lter

In [ ]:
lter.to_sql(name='sample', con=con, if_exists='append', index=False)

2) Joy-Warren 2019 dataset
- Link to source and info

In [ ]:
"""Writes Joy-Warren 2019 dataset to database"""
joyw: DataFrame = pd.read_csv('../data/in/datasets/modified/joy_warren.csv', encoding='utf-8')
# group by depth and station: station1([1.7, 1.8, 2.1][9.7. 9.8, 10.2]...), station2...
joyw = joyw.groupby([joyw["depth"].pct_change().abs().gt(0.15).cumsum(), "station"]).mean(numeric_only=True)
joyw = joyw.reset_index(level=0, drop=True).reset_index(level=0).sort_values(by='depth')
joyw.sort_values(by=['station', 'depth'])
# --------------------------------------------------
jwchemtax = pd.read_csv('../data/in/datasets/modified/joy_warren_chemtax.csv')
jwchemtax = jwchemtax.dropna().sort_values(by='depth')
# --------------------------------------------------
# Join main sample data with chemtax by matching station and depth
joyw: DataFrame = pd.merge_asof(jwchemtax, joyw, by='station', on='depth', direction='nearest',
                                  tolerance=2).sort_values(by='station')

joyw['timestamp'] = pd.to_datetime(joyw['date'], format='%Y%m%d', errors='coerce').dropna().drop(
    columns=['date', 'time'])
joyw['source_name'] = 'joyw'
joyw['percent_phaeo'] = joyw['haptophytes']
joyw['percent_diatom'] = joyw['diatoms']
joyw['percent_other'] = joyw['chlorophytes'] + joyw['mixed_flagellates'] + joyw['cryptophytes']
# ----------------------------------------------------
# Set id field for sample data so foreign keys for microscopy can be matched
joyw = joyw.reset_index(drop=True)
max_id: int = pd.read_sql("select max(id) from sample", con=con)['max(id)'][0] + 1
joyw['id'] = np.arange(max_id, max_id + len(joyw))
jwmkey = pd.concat([joyw['id'], joyw['station'], joyw['depth']], axis=1).sort_values(by='depth')
# Group extra columns into JSON
extra = joyw.columns.difference(sophysql.get_table_cols("sample"))
joyw["extra_json"] = joyw[extra].agg(lambda r: r[r.notna()].to_json(), axis=1)
joyw = joyw.drop(columns=extra, axis=1)

microscopy = pd.read_csv('../data/in/datasets/modified/joy_warren_microscopy.csv', encoding='utf-8').dropna()
replace: tuple = ('centric', 'pennate', 'unknown diatom', 'dinoflagellate', 'ciliate', 'silicoflagellate')
# are_taxa = rows that have species name we can get taxonomy for (ex: Phaeocystis)
are_taxa = ~microscopy['taxa'].isin(replace)
taxa: DataFrame = pd.read_csv("../data/in/worms/joy_warren_worms.csv", encoding='utf-8').rename(sophytaxa.worms_sql)
taxa.index = microscopy[are_taxa].index
# ----------------------------------
microscopy['aphia_id'] = taxa['AphiaID']
microscopy = microscopy.sort_values(by='depth')
# Join microscopy data with matching id in sample table (by depth and station)
microscopy = pd.merge_asof(microscopy, jwmkey, by='station', on='depth', direction='nearest', tolerance=1)
microscopy = microscopy.rename({'id': 'sample_id', 'taxa': 'name', 'group': 'groups'},
                               axis="columns")
# Remove extra columns
microscopy = microscopy[microscopy.columns.intersection(sophysql.get_table_cols("microscopy"))]

data_gdf = GeoDataFrame(joyw, geometry=gpd.points_from_xy(joyw['longitude'], joyw['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(geolabel.zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labels for zones and sectors
joyw = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right'])).drop_duplicates(subset=['id'])
sectors_series: pd.Series = pd.cut(joyw['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
joyw = joyw.assign(sector=sectors_series)
joyw

In [ ]:
joyw.to_sql(name='sample', con=con, if_exists='append', index=False)
microscopy.to_sql(name='microscopy', con=con, if_exists='append', index=False)

3) Phytobase
- Link to source and info

In [ ]:
phybase: DataFrame = pd.read_csv('../data/in/datasets/mod/phytobase.csv')
phybase['timestamp'] = pd.to_datetime(phybase['timestamp'], errors='coerce')
# get full taxonomy of microscopy data as dataframe
taxa: DataFrame = pd.read_csv('../data/in/worms/phytobase_worms.csv')[['original', 'AphiaID']]
# join on sample and taxonomy (by aphia_id), only keep cols in the occurrence table (filter out order, genus, etc)
phybase = pd.merge(phybase, taxa, left_on='scientificname', right_on='original')
phybase = phybase.rename(columns=sophytaxa.worms_sql).filter(sophysql.get_table_cols("occurrence"))

data_gdf = GeoDataFrame(phybase, geometry=gpd.points_from_xy(phybase['longitude'], phybase['latitude']), crs='EPSG:4326')
data_gdf = data_gdf.to_crs(crs=ccrs.SouthPolarStereo())
zones_gdf = gpd.read_file(geolabel.zones_shapefile).to_crs(ccrs.SouthPolarStereo())
# Spatially join data points with zones (polygons) to get labelled data
phybase = DataFrame(data_gdf.sjoin(zones_gdf, how='left').drop(columns=['geometry', 'index_right']))
sectors_series: pd.Series = pd.cut(phybase['longitude'], bins=[-180, -130, -60, 20, 90, 160, 180],
                                    labels=['Ross', 'BA', 'Weddell', 'Indian', 'WPO', 'Ross'], ordered=False)
phybase = phybase.assign(sector=sectors_series)
phybase

In [ ]:
phybase.to_sql(name='occurrence', con=con, if_exists='append', index=False)

4) Alderkamp dataset
- Link to source and info

In [ ]:
pd.read_sql("select * from sample;", con=con)

5) Garibotti dataset
- Link to source and info